In [1]:
from datetime import datetime
from dateutil.parser import parse
from __future__ import division
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import sys

%matplotlib inline

## Data loading

In [2]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
orders = pd.read_csv("X_train.csv")
orders_label = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

# 1 Exploration des données
Puisque on dispose de trois séries de données, dont deux ont un nombre important de colonnes, on préfère les examiner séparément.
Dans cette phase on vise à connaître la façon dont les données sont organisées et à decouvrir éventuels erreurs dans leur représentation. 

## 1.1 Table Customers

In [3]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,14089083.0,SE,1979-02-05 00:00:00,Femme,2013-03-16 23:00:05
1,12862066.0,FR,1982-08-04 00:00:00,Femme,2012-02-14 17:47:33
2,14791699.0,FR,1965-04-02 00:00:00,Femme,2013-10-04 23:10:42
3,10794664.0,FR,1966-04-09 00:00:00,Femme,2010-03-25 18:46:59
4,15268576.0,ES,1980-04-22 00:00:00,Femme,2014-03-19 10:48:39


La table customers est constituèe par les cinq colonnes colonnes suivantes : « CustomersId », « CountryISOCode », « BirthDate », « Gender », « FirstOrderDate ».

In [4]:
customers.dtypes

CustomerId        float64
CountryISOCode     object
BirthDate          object
Gender             object
FirstOrderDate     object
dtype: object

Grâce au command précédent, on peut observer que le type de la colonne « CustomersId » est à tort considerè comme un nombre, et que les colonnes contenant des dates devront être transformè en object de type 'datetime', puisque pour l'instant ils sont considerèes comme objects.

In [5]:
customers.isnull().sum()

CustomerId        0
CountryISOCode    0
BirthDate         0
Gender            0
FirstOrderDate    0
dtype: int64

Heureusement, aucune colonne de la table Customers contient des valeurs nulls. 

In [ ]:
customers_contvars = customers.describe().columns
customers_contvars

Index(['CustomerId'], dtype='object')

In [ ]:
customers_catvars = customers.describe(include=["object"]).columns
customers_catvars

L'analyse des colonnes continues et catégoriques confirme ce que on avait déjà trouvé concernant la colonne 'CustomerId', qui est considerèe comme continue.

## 1.2 Table Products

In [ ]:
products.head()

La table Products, en revanche, est constituèe par un nombre plus elevè de colonnes (25), qui nous donnent des informations concernant les produit vendus.  

In [ ]:
products.dtypes

Les colonnes « VariantId », « BrandId », « ProductId », « ProductColorId » et  « IsNewCollection », comme on a déjà vu dans la table 'Customers', sont à tort considerèe comme numerique, et devront être transformèe en categorique (ou directement supprimèe, comme elles n'ajoutent pas d'information utile
dans notre intérêt).

In [ ]:
products.isnull().sum()

D'ailleurs, dans la table 'Products' on peut retrouver plusieurs colonnes qui contiennent un grand nombre de valeurs nulls: on va ainsi inspecter ces colonnes afin de savoir si les valeurs nulls peuvent être considerèes comme des zeros ou s'ils constituent vraiment des donnèes manquantes.

On commencera par la colonne « ProductType ».

In [ ]:
products['ProductType'].drop_duplicates()

Tout d'abord, on peut observer que en vrai nos donnèes concernent soit des chaussures (pour la plupart) soit autres produits, comme sacs, ceintures, etc...

Puisque les autres colonnes ayants des valeurs nulls sont spécifiques pour les chaussures (« CalfTurn », « UpperHeight », « HeelHeight », « OutSoleMaterialLabel », « RemovableSole », « SizeAdviceDescription »), on veut decouvrir si en effet les valeurs nulls correspondent à produits qui ne sont pas des chaussures. 

On va observer les colonne « CalfTurn » et « UpperHeight ».

In [ ]:
nochauss = products.loc[products['CalfTurn'].isnull()]
nochauss['ProductType'].drop_duplicates()

In [ ]:
nochauss = products.loc[products['UpperHeight'].isnull()]
nochauss['ProductType'].drop_duplicates()

On a ainsi decouvert que les valeurs nulls dans ces colonnes sont dus au fait que pour le produit consideré (soit-il une chaussure ou pas) cette valeur n'est pas mesurable.

In [ ]:
products.describe()

Par rapport seulement aux colonnes numeriques ( « MinSize », « MaxSize », « CalfTurn », « UpperHeight », « HeelHeight » et « PurchasePriceHT » ) on peut observer que:

- On a plusieurs valeurs de 0 dans certains colonnes, qu'on devra decider comme traiter comparé aux valeurs nulls présents dans les memes colonnes.

- Certaines valeurs mins et max ne sont pas consistantes (par exemple, 1 et 720 comme valeurs max et min des colonnes MaxSize et MinSize, alors qu'on aurait espéré valeurs entre 35 et 48)

- Certaines valeurs semblent clairement des outliers ( « HeelHeight » 113, par exemple)

In [ ]:
products_contvars = products.describe().columns
products_contvars

In [ ]:
products_catvars = products.describe(include=["object"]).columns
products_catvars

L'analyse des colonnes continues et catégoriques confirme les erreurs dans les colonnes déjà mentionnées.

En outre, la colonne « SizeAdviceDescription » semble interessant si on arrive à en extraire des informations.

In [ ]:
products['SizeAdviceDescription'].drop_duplicates()

Meme si les donnèes sont de type chaîne, il y a peu de valeurs differents, ce qui nous permettra heureusement d'en extraire des bonnes features.

## 1.2 Table Orders

In [ ]:
orders.head()

Enfin, la table Orders est constituèe par 20 colonnes et contient les informations concernant les ordres d'achat. Ces données sont du meme type que celles qui nous sont données pour le test: on devra ainsi réaliser les memes operations sur tous les deux groupes de données.

In [ ]:
orders.dtypes

Comme dans les tables precedentes, aussi ici on retrouve des colonnes qui ne devraient pas être numerique mais plutôt categoriques ( « OrderNumber », « VariantId », « LineItem », « CustomerId », « OrderNumCustomer » et « IsOnSale » ).

In [ ]:
orders.isnull().sum()

Heureusement, les colonnes contenant des valeurs nulls sont seulement deux. 
Notamment, pour la colonne « IsOnSale » on peut très bien supposer que les valeurs nulls soient equivalentes à des zeros (produits non en offre).

In [ ]:
orders.describe()

En ce cas, le command 'describe' nous donne des informations seulement à propos de la colonne « Quantity », dont on peut observer des valeurs consistantes, puisque toutes les autres colonnes ne sont pas à considerer comme numeriques, comme déjà remarquè.

In [ ]:
orders_contvars = orders.describe().columns
orders_contvars

In [ ]:
orders_catvars = orders.describe(include=["object"]).columns
orders_catvars